In [13]:
# %cd f-threshold-select

In [14]:
# !git clone https://github.com/fabian-sp/GGLasso.git

In [1]:
import gglasso
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import scipy as sp
import matplotlib.pyplot as plt

from numpy.linalg import matrix_rank
from matplotlib.pyplot import figure
from scipy import stats
from scipy.linalg import eigh
from numpy import genfromtxt

from gglasso.solver.admm_solver import ADMM_MGL
from gglasso.problem import glasso_problem

from gglasso.helper.data_generation import generate_precision_matrix, group_power_network, sample_covariance_matrix
from gglasso.helper.basic_linalg import adjacency_matrix
from gglasso.helper.data_generation import time_varying_power_network, sample_covariance_matrix
from gglasso.helper.experiment_helper import lambda_grid, discovery_rate, error
from gglasso.helper.utils import get_K_identity
from gglasso.helper.experiment_helper import plot_evolution, plot_deviation, surface_plot, single_heatmap_animation
from gglasso.helper.model_selection import aic, ebic, K_single_grid

### Read data

In [2]:
sub_corr = []

for i in range(0, 50):
    sub_corr.append(genfromtxt("/storage/groups/bds01/datasets/brains/sub_corr50/sub_corr{0}.csv".format(i), delimiter=','))

sub_corr = np.array(sub_corr)
sub_corr.shape

(50, 436, 436)

### Single GL

In [7]:
lambda1_range = np.logspace(0, -1, 5)
mu1_range = np.logspace(0, -1, 5)

In [5]:
N = sub_corr.shape[1]

In [11]:
est_uniform, est_indv, statistics = K_single_grid(sub_corr[:1, :], lambda1_range, N,
                                                  mu_range = mu1_range,
                                                  method = 'eBIC', gamma = 0.3, 
                                                  latent = True, use_block = True)

------------Range search for instance 0------------
ADMM terminated after 52 iterations with status: optimal.
ADMM terminated after 89 iterations with status: optimal.
ADMM terminated after 102 iterations with status: optimal.
ADMM terminated after 123 iterations with status: optimal.
ADMM terminated after 258 iterations with status: optimal.
ADMM terminated after 132 iterations with status: optimal.
ADMM terminated after 88 iterations with status: optimal.
ADMM terminated after 103 iterations with status: optimal.
ADMM terminated after 124 iterations with status: optimal.
ADMM terminated after 267 iterations with status: optimal.
ADMM terminated after 135 iterations with status: optimal.
ADMM terminated after 88 iterations with status: optimal.
ADMM terminated after 102 iterations with status: optimal.
ADMM terminated after 125 iterations with status: optimal.
ADMM terminated after 270 iterations with status: optimal.
ADMM terminated after 144 iterations with status: optimal.
ADMM ter

In [16]:
K = est_uniform["Theta"].shape[0]

In [17]:
# dump matrices into csv
for i in range(0, K):
    np.savetxt("/storage/groups/bds01/datasets/brains/est_uniform_latent50/Theta{0}.csv".format(i), est_uniform["Theta"][i], 
               delimiter=",", header='')
    np.savetxt("/storage/groups/bds01/datasets/brains/est_uniform_latent50/L{0}.csv".format(i), est_uniform["L"][i], 
               delimiter=",", header='')

In [18]:
!jupyter nbconvert --to script --no-prompt SGL_latent.ipynb

[NbConvertApp] Converting notebook SGL_latent.ipynb to script
[NbConvertApp] Writing 2380 bytes to SGL_latent.py


In [56]:
with open('/storage/groups/bds01/datasets/brains/statistics_SGL_latent50.txt', 'w') as f:
    print(statistics, file=f)

### Read solution

In [19]:
Theta = []
L = []

for i in range(0, 50):
    Theta.append(genfromtxt("/storage/groups/bds01/datasets/brains/est_uniform_latent50/Theta{0}.csv".format(i), delimiter=','))
    L.append(genfromtxt("/storage/groups/bds01/datasets/brains/est_uniform_latent50/L{0}.csv".format(i), delimiter=','))

Theta, L = np.array(Theta), np.array(L)
Theta.shape, L.shape

OSError: /storage/groups/bds01/datasets/brains/est_uniform_latent50/Theta1.csv not found.